# Customer Segmentation for Marketing
### Scenareo
Lets assume you are a data analyst in a company. The sales team in your company has a target they want to hit in the next quater. It is the highest target number they have ever had. They believe, to hit the number they have to break the norm of how they aproach customers and create a new strategy that will yield to more conversions. After all, as my fried says, only a mad mad does the same thing over and over again expecting  some different results.
The team believes, by better understanding the individual behaviours of the customers they can know who to invest energy on and who will buy even without much involvement. this level of information on the custoer will help the team approach the customers with relevant offers amd messages that applies to them.

The sales manage walk to your desk for assistance. After analyzing the scenareo, you decide the solution would be to create a dashbord for the team to segment the customer according to their purchase behaviour. The dashboard is to help understand the cartegory of each customer and hens the level of engagement with each customer.

#### Methodology
- Predict if the customer is likely to make a purchase in the next 3 months. Determine if the customer is alive or has aready churned
- Predict the potential sales volume of the customers in the next 3 months. Is the customer likely to make high or low cumulative volume of sale in the next 3 month

##### Target columns
- customer id
- Recency
- Frequency
- Monitory Value
- CLV
- Predicted avg sales
- Predicted no. of purchases
- Predicted total sales
- Prob of cust being alive

#### Dataset
In this study, we are going to use a free online retail dataset from the Uci Machine Learning Repository.This is a transactional data set that contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

#### Objectives
To create a script that transforms the row transactions data into a custorers dataset with our variables of interest (probability of a customer making a future purchase, predicted total amount of sales)

In [42]:
## Loading the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
plt.style.use("fivethirtyeight")


### lifetime package for computing CLV
import lifetimes

In [3]:
### Loading the dataset
data=pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
data.shape

(541909, 8)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


inspecting form missing values

In [10]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

We have a total of 135080 customers that we can not indentify. 

### Prove why this customers need to be droped
since we are interested in studying the purchase pattern of each customers. The customers we can not recognize would not provide any recognizable trend. So, we just have to drop them

In [11]:
data=data[pd.notnull(data['CustomerID'])]
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

#### Selecting the relevant features
For computing the Lifetime value of the customers, we dont need all the attributes in the dataset. we only need the customerid, invoice date, invoice quantity and unit price

In [30]:
clv_data=data[['CustomerID','InvoiceNo','Quantity', 'InvoiceDate',
       'UnitPrice']]
clv_data['Revenue']=data['Quantity']*data['UnitPrice']

clv_data.sample(3)

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Revenue
332882,16515.0,566124,6,2011-09-09 11:15:00,1.25,7.5
174986,12540.0,551868,12,2011-05-04 16:29:00,1.65,19.8
335035,16549.0,566275,2,2011-09-11 13:58:00,0.55,1.1


Exploring the distridution of the dataset

In [31]:
clv_data[['Quantity', 'Revenue','UnitPrice']].describe()

,Quantity,Revenue,UnitPrice
count,406829.000000,406829.000000,406829.000000
mean,12.061303,20.401854,3.460471
std,248.693370,427.591718,69.315162
min,-80995.000000,-168469.600000,0.000000
25%,2.000000,4.200000,1.250000
50%,5.000000,11.100000,1.950000
75%,12.000000,19.500000,3.750000
max,80995.000000,168469.600000,38970.000000


The Total Sales and Quantity variables have negative minimum values. This could be caused by return items or Cancelation. Since we are interested in computing the total value a customers can contribute to the business in his lifetime, we can drop all the negative values

In [32]:
clv_data=clv_data.query("Revenue>0")
clv_data.describe()

,CustomerID,Quantity,UnitPrice,Revenue
count,397884.000000,397884.000000,397884.000000,397884.000000
mean,15294.423453,12.988238,3.116488,22.397000
std,1713.141560,179.331775,22.097877,309.071041
min,12346.000000,1.000000,0.001000,0.001000
25%,13969.000000,2.000000,1.250000,4.680000
50%,15159.000000,6.000000,1.950000,11.800000
75%,16795.000000,12.000000,3.750000,19.800000
max,18287.000000,80995.000000,8142.750000,168469.600000


Finalizing on data cleaning.

- Remove time from invoce date to remain with just date.
- Conver Customer ID to a cartegorical variable

In [33]:
clv_data['InvoiceDate']=clv_data['InvoiceDate'].dt.date
clv_data['InvoiceDate']=pd.to_datetime(clv_data['InvoiceDate'])
clv_data['CustomerID']=clv_data['CustomerID'].astype(object)

clv_data.describe(include='O').T

,count,unique,top,freq
CustomerID,397884.0,4338.0,17841.0,7847.0
InvoiceNo,397884.0,18532.0,576339.0,542.0


our dataset has 4338 unique customers. 

In [34]:
clv_data['InvoiceDate']=pd.to_datetime(clv_data['InvoiceDate'])
clv_data.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Revenue
0,17850.0,536365,6,2010-12-01,2.55,15.30
1,17850.0,536365,6,2010-12-01,3.39,20.34
2,17850.0,536365,8,2010-12-01,2.75,22.00
3,17850.0,536365,6,2010-12-01,3.39,20.34
4,17850.0,536365,6,2010-12-01,3.39,20.34


## Computing Customer Lifetime Value using LIFETIMES module

### Theory and assumptions of BG/NBD
### Theory of RFM
This method uses the RFM table to compute different the customer lifetime value for eith a grop of customers or individual customers lifetime value in the business.
- Recency - Time elapsed since the last purchase
- Frequency - The number of repeat purchases
- Monitory Value - Average each customers purchase value
- T - Time between the customers first purchase and the very last purchase in the dataset

In [46]:
clv_summary=lifetimes.utils.summary_data_from_transaction_data(clv_data,'CustomerID','InvoiceDate','Revenue',observation_period_end=clv_data['InvoiceDate'].max(),freq='D').reset_index()
clv_summary.sample(5)

AttributeError: 'DataFrame' object has no attribute 'ix'